# Derivations related to Least Square Error (LSE) 

### What is Least Square Error (LSE)?

LSE is defined as the sum of squared residuals of a dataset. To put this in perspective, let's take
$x^{(i)}$ and $y^{(i)}$ an input vector and label respectively, and our prediction: $y_{pred}^{(i)}$ after running our model on the input $x^{(i)}$

Then we have a residual (or error) of a specific example as $(y^{i} - y_{pred}^{i})$ which is the difference between the actual output and our predicted output.

Now we need to square the residuals. $(y^{i} - y_{pred}^{i})^{2}$
There are several reasons we do this, but I will mention two here:
1. If the difference between actual and predicted is negative i.e. $(y^{i} - y_{pred}^{i}) < 0 $ then by squaring the term, we always receive a positive outcome, so the differences are treated simply by their magnitude rather than their magnitude and sign

1. By squaring the term, larger differences are valued *MORE* than smaller differences since squaring a term is non-linear

Finally, we need to sum over each data point (example) to obtain the total error over all data points:
$$\\ LSE = \sum_{i}(y^{i} - y_{pred}^{i})^{2}$$

### Coding the LSE Formula

In [31]:
import numpy as np
import sys
from pathlib import Path

# Add the directory containing your code file to sys.path
sys.path.append(str(Path().resolve().parent.parent))

from models.LinearRegression import LinearRegression

# CREATE SOME DUMMY DATA
x = np.linspace(0, 10, 10)
y = x + 2 + np.random.rand() - 0.5 # y is a linear function of x with some noise

# reshape the data to place into the model
X = np.reshape(x, (x.shape[0], 1))

# initialize our model (I will use my Linear Regression here)
model = LinearRegression(1, 1)

Coding this formula is not very tricky. We first need to calculuate $y_{pred}^{i}$. This is done so by our model, (likely Linear Regression). To do so we execute the following for a vector of inputs $x^{(i)}$. We can think of this as a matrix $X$ where the rows represent each example, and the columns are each feature of the input:

In [32]:
y_pred = model(X)
y_pred

array([0.37951989, 0.65422855, 0.92893722, 1.20364588, 1.47835455,
       1.75306321, 2.02777188, 2.30248054, 2.5771892 , 2.85189787])

The output of this function is a vector $y_{pred}$ where each entry is the $y_{pred}^{(i)}$ for the corresponding $x^{(i)}$
<br> We now need to take our actual outputs $y$ and element-wise subtract the $y_{pred}$ vector. 
<br> We can do this easily if both y and X are numpy arrays.
<br>Then we need to square each element and sum all of the elements together. (Since the subtraction of each output by the prediction results in a vector of residuals) 
<br>Thus the entire function is defined as:

In [33]:
def LSELoss(y_pred, y):
    return np.sum((model(X) - y) ** 2)

print(f"Loss is: {LSELoss(y_pred, y)}")

Loss is: 393.11852014845823


### Why Least Squares? (Digging into the theory)
##### *(Skip to the end for a high-level summary without all of the math)*
Assume we have the same circumstances as above with $y_{pred}^{(i)}, y^{(i)}, x^{(i)}$

Now we can relate our predicted output and actual output as the following:
$\\ y^{(i)} = y_{pred}^{(i)} + \epsilon^{(i)}$, where $\epsilon^{(i)} \sim \mathcal{N}(0, \sigma^{2})$
This means that our predicted output + some error (unmodelled effects, noise, etc.) is equal to our actual output,
for each example. We are assuming that our unmodelled error is normally distributed, and that each $\epsilon^{i}$ is *i.i.d. (Independent and Identically Distributed)*

By assuming this, we now have a probability model of our unmodelled error:
$$ P(\epsilon^{(i)}) = \frac{1}{\sqrt{2\pi}\sigma}exp[-\frac{\epsilon^{(i)^{2}}}{2\sigma^{2}}]$$

The probability of this specific unmodelled error occuring (since we are assuming is a normal r.v.) is the same as the probability of our output occuring, given our input and using the formula relating our output to predicted output, we can replace our unmodelled error:

$$ P(y^{(i)} | x^{(i)}) = \frac{1}{\sqrt{2\pi}\sigma}exp[-\frac{(y^{(i)} - y_{pred}^{(i)})^{2}}{2\sigma^{2}}] \implies  y^{(i)} | x^{(i)} \sim \mathcal{N}(y_{pred}^{(i)}, \sigma^{2})$$

So now we know that our output follows a normal distribution with an expectation of our predicted output, meaning that 
we can find a likelihood function of our predicted output throughout all $m$ examples. Since $y_{pred}^{(i)}$ is a function of our parameters we define our Likelihood function as a function of our parameters, rather than $y_{pred}^{(i)}$. Let $$g^{(i)}: \mathbb{R}^{n} \rightarrow \mathbb{R} \\ \theta \rightarrow y_{pred}^{(i)}$$

This is just so that our predicted output is defined as a function of our parameter vector $\theta$
<br> Then we have the Likelihood as:

$$ \mathcal{L}(\theta) = \prod_{i = 1}^{m} P(y^{(i)} | x^{(i)}) $$

By taking the log of the likelihood, we then get a summation:

$$ \log{\mathcal{L}(\theta)} = l(\theta) = \sum_{i=1}^{m}\log{\frac{1}{\sqrt{2 \pi \sigma}}} + \log{exp[-\frac{(y^{(i)} - g^{(i)}(\theta))^{2}}{2\sigma^{2}}]}
\\ = m\log{\frac{1}{\sqrt{2 \pi \sigma}}} + \sum_{i=1}^{m}-\frac{(y^{(i)} - g^{(i)}(\theta))^{2}}{2\sigma^{2}}$$

Now by using Maximum Likelihood Estimation we need to choose the parameters $\theta$ of our model to maximize $l(\theta)$

$$ \underset{\theta}{\mathrm{argmax }} [m\log{\frac{1}{\sqrt{2 \pi \sigma}}} + \sum_{i=1}^{m}-\frac{(y^{(i)} - g^{(i)}(\theta))^{2}}{2\sigma^{2}}]
\\ = \underset{\theta}{\mathrm{argmax}} [\sum_{i=1}^{m}-\frac{(y^{(i)} - g^{(i)}(\theta))^{2}}{2\sigma^{2}}]
\\ = \underset{\theta}{\mathrm{argmin}} [\sum_{i=1}^{m}(y^{(i)} - g^{(i)}(\theta))^{2}]
\\ = \underset{\theta}{\mathrm{argmin}} [LSE]
$$

Ok so, at a high level, if we assume that our outputs follow a normal distribution, centered around our predicted output (our predicted output should be the average of the real outputs), then by using maximum likelihood estimation, we end up minimizing the least sqaures error to find our optimal parameters.



### The Derivative of LSE

In order to perform gradient descent or any other optimization algorithm, we require the derivative of the loss function w.r.t. each parameter in the model, or the gradient of the loss function. Let's define the model as $g^{(i)}(\theta)$ where $i$ is referencing the training example. I.e. we make a function for each training example (fix $x^{(i)}$) and make the $\theta = (\theta_{1}, ..., \theta_{n})$ vector the variable.

Let $J(\theta) = LSE$

Then $$\frac{\partial J}{\partial \theta_{j}} = \frac{\partial}{\partial \theta_{j}}[\sum_{i=1}^{m}(y^{(i)} - g^{(i)}(\theta))^2]$$

Since each example in the summation includes $g^{(i)}(\theta)$ which is a function of $\theta_{j}$, we cannot discard the summation and must take the derivative of the inside of the summation.

$$\frac{\partial J}{\partial \theta_{j}} = \sum_{i=1}^{m}\frac{\partial}{\partial \theta_{j}}(y^{(i)} - g^{(i)}(\theta))^2$$

Then using the chain rule,
$$ = \sum_{i=1}^{m}2 \cdot (y^{(i)} - g^{(i)}(\theta)) \cdot \frac{\partial g^{(i)}}{\partial \theta_{j}} $$

This is the complete partial derivative w.r.t a single parameter in the model, however we can make things simpler to code by transforming this into vectors and matrices allowing for the entire gradient to be computed at the same time.
We start by considering that $y^{(i)} - g^{(i)}(\theta)$ is occuring for each training example. This means we can convert it into a vector. 

$$\text{Let  } y = (y^{(1)}, y^{(2)}, ..., y^{(m)}) \text{  and  } g(\theta) = (g^{(1)}(\theta), g^{(2)}(\theta), ..., g^{(m)}(\theta))$$

Now, since we are multiplying each term in the summation by $\frac{\partial g^{(i)}}{\partial \theta_{j}}$, which is a different derivative (because each $g^{(i)}$ is its own function) we can also convert this to a vector:

$$\text{Let } \frac{\partial g}{\partial \theta_{j}} = (\frac{\partial g^{(1)}}{\partial \theta_{j}}, \frac{\partial g^{(2)}}{\partial \theta_{j}}, ..., \frac{\partial g^{(m)}}{\partial \theta_{j}}) $$

Then since we are summing over a product, we can treat this as the inner product (dot product) of two vectors (since this is the definition of the dot product). This turns the awkward summation into a simple inner product calculation

$$\frac{\partial J}{\partial \theta_{j}} = 2 \cdot \frac{\partial g}{\partial \theta_{j}}^{T}(y - g) $$

Now to calculate the full gradient (as this is currently just an arbitrary partial derivate but we want the vector of partial derivatives), we can then consider placing the vector $\frac{\partial g}{\partial \theta_{j}}$ into columns of a matrix for every entry of $\theta$

$$\text{Let } \frac{\partial g}{\partial \theta} = (\frac{\partial g}{\partial \theta_{1}} \rightarrow \frac{\partial g}{\partial \theta_{n}}) $$

This is also known as the Jacobian matrix of $g$. 
<br> Then we can calculate all partials (the gradient vector) simultaneously as:
$$ \nabla_{J}(\theta) = 2 \cdot \frac{\partial g}{\partial \theta}^{T}(y - g)

### Implementing the derivative into code

In [34]:
import numpy as np
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent.parent))

from models.LinearRegression import LinearRegression

# CREATE SOME DUMMY DATA
x = np.linspace(0, 10, 10)
y = x + 2 + np.random.rand() - 0.5 # y is a linear function of x with some noise

# reshape the data to place into the model
X = np.reshape(x, (x.shape[0], 1))

# initialize our model (I will use my Linear Regression here)
model = LinearRegression(1, 1)

Implementing the gradient is fairly easy, after simplifying the calculation down to a matrix vector product. Let's first compute g by running a forward pass through our algorithm.

In [ ]:
y_pred = model(X)
y_pred

Then we need to compute $2 \cdot (y - g)$ (since the Jacobian matrix of the model depends specifically on the model function we will do that after).

In [38]:
def grads(y_pred, y):
        return 2 * (y_pred - y)

loss_grads = grads(y_pred, y)
loss_grads

array([ -3.20018714,  -4.87299203,  -6.54579692,  -8.21860182,
        -9.89140671, -11.56421161, -13.2370165 , -14.90982139,
       -16.58262629, -18.25543118])

Finally, we multiply this by the Jacobian of the model. However, since this is model specific this is done by the model class itself.
<br> For Linear Regression, the Jacobian consists only of the training example inputs, so we pass the input matrix X into the function

In [39]:
final_grads = model.grads(X, loss_grads)
final_grads

array([-689.73090653, -107.27809159])

As you can see we are left with a vector of gradients corresponding to each of the parameters of the model. We can then pass those gradients into an optimization algorithm such as stochastic gradient descent to apply them to the parameters.